### Instalar bibliotecas

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install twitchAPI
#doc https://pytwitchapi.readthedocs.io/

In [ ]:
!python --version

### Preparar Twitch Client

In [ ]:
from dotenv import load_dotenv
import os
import json
import pandas as pd

#Carrega variaveis de ambiente para parametrização
load_dotenv('vars.env')


In [ ]:
#Para obter as autenticações deve-se ter um aplicativo cadastrado em https://dev.twitch.tv/console
#Acessar o nome do aplicativo criado algo similar a https://dev.twitch.tv/console/apps/**********************
#Copiar os campos ID do cliente e Segredo de cliente


#documentacao das infos possivel de obter da api https://dev.twitch.tv/docs/api/reference

#listas de todas as tags
#https://www.twitch.tv/directory/all/tags

from twitchAPI.twitch import Twitch

# create instance of twitch API and create app authentication
twitch = Twitch(os.environ.get('cli_id'), os.environ.get('cli_token'))

#lista de canais (arquivo all_broadcaster_dict.txt) de Feb de 2015 
#obtidos em https://www.dropbox.com/s/vsbzeh69ddjpqx6/broadcaster.rar?dl=0&file_subpath=%2Fall_broadcaster_dict.txt
#fonte: https://clivecast.github.io/
#The file "all_broadcaster_dict.txt" is for academic use only as the source determined

In [ ]:
#O arquivo all_broadcaster_dict.txt tem 2.388.705 registros e demorou mais de 160 horas para extrair toda a base de canais
#recomenda-se dividir o a lista de canais em arquivos menos para o processamento
#O uso do google colab tem a limitação de 6 horas de tempo de execução, no entando após 3 horas de execução o ambiente verifica a inatividade
#foi preciso rodar esse trecho mais de uma vez pois as vezes as consultas dão timeout depois de obter 100mil a 50mil registros
with open('/content/drive/MyDrive/Colab Notebooks/twitch_tcc/all_broadcaster_dict.txt', 'r') as f:
    lista_broadcaster_id = [line.strip() for line in f]

df_brocast = pd.DataFrame({'broadcaster_id': pd.Series(dtype='str'),
                              'broadcaster_login': pd.Series(dtype='str'),
                              'broadcaster_name': pd.Series(dtype='str'),
                              'broadcaster_language': pd.Series(dtype='str'),
                              'game_id': pd.Series(dtype='str'),
                              'game_name': pd.Series(dtype='str'),
                              'title': pd.Series(dtype='str'),
                              'delay': pd.Series(dtype='int'),
                              })
#Cria o arquivo
#Comentar esta linha com o "mode=w" caso re-utilize esse trecho de código, se não ira sobreescrever o que já foi processado
df_brocast.to_csv('/db/base_canais.csv', mode='w', sep=';', header=True)
for broadcaster_id in lista_broadcaster_id:
  try:
    resultado_consulta = twitch.get_channel_information(broadcaster_id=f'{broadcaster_id}')
    if resultado_consulta['data'] != []:
      data_resultado_consulta = pd.DataFrame(resultado_consulta['data'])
      data_resultado_consulta.to_csv('/db/base_canais.csv', mode='a', sep=';', header=False)
  except:
    print(f"Não foi possível obter dados de: {broadcaster_id}")